In [104]:
import polars as pl
import altair as alt
import random

In [105]:
eobs = {
    "date": ["2021-01-04","2021-01-05","2021-01-06","2021-01-07","2021-01-08","2021-01-11","2021-01-12","2021-01-13","2021-01-14","2021-01-15","2021-01-19","2021-01-20"],
    "vti": [190.048508,191.583771,193.307281,196.407532,197.358429,196.179733,196.873093,197.091003,196.902817,195.189224,196.972137,199.458298],
    "vb": [190.040939,192.759628,198.970963,202.255219,202.175842,202.175842,205.192230,203.961853,206.521790,203.763412,206.114990,207.365189]
}
esyn = {
    "date": ["2021-01-04","2021-01-05","2021-01-06","2021-01-07","2021-01-08","2021-01-11","2021-01-12","2021-01-13","2021-01-14","2021-01-15","2021-01-19","2021-01-20"],
    "vti": [190.048508,191.583771,193.307281,196.407532,197.358429,196.179733,196.873093,197.091003,196.902817,195.189224,196.972137,199.458298],
    "vb": [190.040939,192.759628,198.970963,202.255219,202.175842,202.175842,205.192230,203.961853,206.521790,203.763412,206.114990,207.365189]
}
odf = pl.DataFrame(eobs)
sdf = pl.DataFrame(esyn)
sdf = sdf.with_columns(pl.col('vti')+random.uniform(-2,2))
sdf = sdf.with_columns(pl.col('vb')+random.uniform(-2,2))

In [106]:
COLOR_OBSERVED = "#000000"
COLOR_SYNTHETIC = "#f28e2b"

In [107]:
def select_axis_endpoints(df: pl.DataFrame,column: str):
    dmin = df.select(pl.min(column)).item()
    dmax = df.select(pl.max(column)).item()
    smin = dmin-((dmax-dmin)*0.1)
    smax = dmax+((dmax-dmin)*0.1)
    return [smin,smax]

In [108]:
def old_scaled_build_numerical_numerical_chart(df: pl.DataFrame,columns: list,domain: list,range: list,color: str):
    return alt.Chart(df.to_pandas()).mark_point(filled=True).encode(
        x = alt.X(columns[0]+':Q',scale = alt.Scale(domain=[domain[0],domain[1]])),
        y = alt.Y(columns[1]+':Q',scale = alt.Scale(domain=[range[0],range[1]])),
        color = alt.ColorValue(color)
    )
def build_numerical_numerical_chart(df: pl.DataFrame,columns: list,domain: list,range: list,color: str):
    return alt.Chart(df.to_pandas()).mark_point(filled=True).encode(
        x = alt.X(columns[0]+':Q',scale = alt.Scale(zero=False)),
        y = alt.Y(columns[1]+':Q',scale = alt.Scale(zero=False)),
        color = alt.ColorValue(color)
    )

In [109]:
def plot_marginal_2d_numerical_numerical(observed: pl.DataFrame,synthetic: pl.DataFrame,columns: list):
    assert len(columns) == 2, "2d plot requires selection of exactly two columns"
    for c in columns:
        assert c in observed.columns, f"Column '{c}' not in observed data"
        assert c in synthetic.columns, f"Column '{c}' not in synthetic data"
        
    observed = observed[columns].with_columns(pl.lit("observed").alias("plot_data_source"))
    synthetic = synthetic[columns].with_columns(pl.lit("synthetic").alias("plot_data_source"))
    combined = pl.concat([observed,synthetic])
    domain = select_axis_endpoints(combined,columns[0])
    range = select_axis_endpoints(combined,columns[1])

    oplot = build_numerical_numerical_chart(combined,columns,domain,range,COLOR_OBSERVED).transform_filter(alt.datum.plot_data_source == "observed")
    splot = build_numerical_numerical_chart(combined,columns,domain,range,COLOR_SYNTHETIC).transform_filter(alt.datum.plot_data_source == "synthetic")
    return oplot+splot

plot_marginal_2d_numerical_numerical(odf,sdf,['vti','vb'])


alt.LayerChart(...)